In [ ]:
# !pip install openai
# !wget https://raw.githubusercontent.com/allenai/bi-att-flow/master/squad/evaluate-v1.1.py
# !pip install sentencepiece
# !pip install peft
# !pip install sacrebleu

In [ ]:
from openai import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline, AutoModelWithLMHead
import torch
from  transformers  import  AutoTokenizer, AutoModelWithLMHead, pipeline
from google.colab import drive
import glob
from transformers.pipelines import pipeline
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
import os
import re
import random
from tqdm.notebook import tqdm
import json
import time
drive.mount('/content/drive')
client = OpenAI(api_key='KEY')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
def do_experiment(path_regex,client,test_fn, result_path, prefix, batch_size):
    for file_path in glob.glob(path_regex):
        file_name = os.path.split(file_path)[-1]
        lang = file_name.split('.')[1]
        results = []
        with open(file_path, 'r', encoding='utf-8') as f:
            xquad_json = json.load(f)
            for data in tqdm(xquad_json['data']):
                title = data['title']
                for paragraph in data['paragraphs']:
                  context = paragraph['context']
                  questions = list(map(lambda x:x['question'], paragraph['qas']))
                  answers = list(map(lambda x:x['answers'][0]['text'], paragraph['qas']))
                  ids = list(map(lambda x:x['id'], paragraph['qas']))
                  for batch_question, batch_ids in zip(batch(questions, batch_size), batch(ids, batch_size)):
                      model_answers = test_fn(context, batch_question, answers, title, client)
                      for model_answer, id in zip(model_answers, batch_ids):
                          results.append({id:model_answer})

        out_path = os.path.join(result_path, prefix + '/' + file_name)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w+", encoding='utf-8') as out_f:
            json.dump(results, out_f, ensure_ascii=False)

In [ ]:
def test_function(context, questions, answers, title, client):
    output = []
    for q_id, (question, answer) in enumerate(zip(questions, answers)):
        if q_id == len(questions) - 1:
              ex_question = questions[0]
              ex_answer = answers[0]
        else:
              ex_question = questions[-1]
              ex_answer = answers[-1]

        prompt = f"Title: {title}\nBackground: {context}\nQ: {ex_question}\nA: {ex_answer}\nQ: {question}\nA: "
        completion = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "user", "content": prompt}])
        output.append(completion.choices[0].message.content)
        time.sleep(0.13)
    return output

In [ ]:
path_regex = '/content/drive/MyDrive/xquad/xquad.zh.json'
do_experiment(path_regex,client,test_function, '/content/drive/MyDrive/', 'GPT_35_TURBO_normal', 32)

  0%|          | 0/48 [00:00<?, ?it/s]